# # Convert RealSense .bag → MP4

In [ ]:
import numpy as np
import pyrealsense2 as rs
import cv2

def bag_to_mp4(bag_path: str, out_path: str, fps: float = 30.0):
    """
    Read a RealSense .bag file and write out a color-only MP4.
    """
    pipeline = rs.pipeline()
    cfg = rs.config()
    cfg.enable_device_from_file(bag_path, repeat_playback=False)
    profile = pipeline.start(cfg)

    # get color stream resolution
    color_stream = profile.get_stream(rs.stream.color).as_video_stream_profile()
    w, h = color_stream.width(), color_stream.height()

    # set up VideoWriter (mp4v codec)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

    try:
        while True:
            frames = pipeline.wait_for_frames()
            color = frames.get_color_frame()
            if not color:
                continue
            # convert to numpy array (RGB)
            img_rgb = np.asanyarray(color.get_data())
            # convert RGB→BGR for correct colors in OpenCV
            img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
            writer.write(img_bgr)
    except RuntimeError:
        # bag playback ended
        pass
    finally:
        writer.release()
        pipeline.stop()
        print(f"✅ Wrote {out_path}")



# Run conversion

In [2]:
bag_to_mp4(bag_file, output_mp4, fps)

✅ Wrote /home/yogee/Desktop/human_detector_ws/training_data/raw_data/video1.mp4
